# Lesson 5 - Assignment

In this assignment, you will implement a Support Vector Machine Classifier  from scratch and compare the results to existing sklearn algorithm. 

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# make this notebook's output stable across runs
np.random.seed(0)

In [2]:
# ChenWei I added this class to see a working example of SVM
# I was having trouble with my own algorithm and this helped me at least a function that worked.
# This comes from AssemblyAI series on ML from scratch
# https://www.youtube.com/playlist?list=PLcWfeUsAys2k_xub3mHks85sBHZvg24Jd
# used this reference code to help understand and debug my implementation
class SupportVectorMachine:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000) -> None:
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None   #weights
        self.b = 0      #biases

    def fit(self, X: np.ndarray, y:np.ndarray, random_weights=False) -> None:
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)

        if random_weights:
            self.w = np.random.uniform(-1.0, 1.0, n_features)
        else:
            self.w = np.zeros(n_features)

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X: np.ndarray):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

Question 1.1: Implement the cost function cost/objective function:

<img src="https://miro.medium.com/max/688/1*JAS6rUTO7TDlrv4XZSMbsA.png" alt="drawing" width="600"/>


In [3]:
# implement the cost/objective function for Support Vector Machine
# reg_strength is C in the above equation
def compute_cost(W, X, Y, regularization_strength=1000):
    # calculate hinge loss
    N = X.shape[0] # number of training examples
    distances = 1 - Y * (np.dot(W, X.T).flatten())
    distances[distances < 0] = 0  # convert anything less than 0 to 0

    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 0.5 * np.dot(W, W)**2 + hinge_loss
    return cost

def compute_cost2(W, X, Y, regularization_strength=1000):
    w_norm = 0.5*np.dot(W, W)**2
    offsets = Y * (np.dot(W, X.T).flatten())
    smax = np.clip(a=1-offsets, a_min=0.0, a_max=np.inf)
    hinge_loss = regularization_strength * np.sum(smax)/len(smax)
    cost = w_norm + hinge_loss
    return cost

Question 1.2: Write a method that calculate the cost gradient:

<img src="https://miro.medium.com/max/866/1*ww3F21VMVGp2NKhm0VTesA.png" alt="drawing" width="600"/>

In [4]:
def calculate_cost_gradient(W, X_batch, Y_batch, regularization_strength=1000):

   dw = np.zeros(len(W))
   N = len(Y_batch)
   # compute the losses up front
   # doing this outside the loop speeds up the gradient calculation
   losses = 1 - Y_batch * (np.dot(X_batch, W))
   losses[losses < 0] = 0  # convert anything less than 0 to 0
   for x, y, hinge_loss in zip(X_batch, Y_batch, losses):
       if hinge_loss == 0:
           dw = W
       else:
           dw = W - (regularization_strength * y * x)

   dw = dw/N  # average
   return dw

def calculate_cost_gradient2(W, X_batch, Y_batch, regularization_strength=1000):
   dw = np.zeros(W.shape[0])
   for x, y in zip(X_batch, Y_batch):
       hinge_loss = np.max((0, 1.0 - np.dot(W.T, x)*y))
       if hinge_loss == 0:
           dw += W
       else:
           dw += W - (regularization_strength * y * x)
   return dw/len(Y_batch)  # average

Question 1.3: Write a method that performs stochastic Gradient descent as follows:
- Calculate the gradient of cost function i.e. ∇J(w)
- Update the weights in the opposite direction to the gradient: w = w — ∝(∇J(w))
- Repeat until conversion or until 5000 epochs are reached

In [5]:
def sgd(data, outputs, learning_rate = 0.0001, max_epochs = 4096, cost_threshold=0.01, random_weights=False):
    print("Training using Stochastic Gradient Descent 1 method")
    if random_weights:
       weights = np.random.uniform(-1.0, 1.0, data.shape[1])
    else:
       weights = np.zeros(data.shape[1])

    nth = 0
    prev_cost = float("inf")
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(data, outputs)
        for x_chunk, y_chunk in zip(np.array_split(X, 100), np.array_split(Y, 100)):
            ascent = calculate_cost_gradient(weights, x_chunk, y_chunk)
            weights = weights - learning_rate * ascent
            # convergence check on 2^nth epoch
            if epoch == 2 ** nth or epoch == max_epochs - 1:
                cost = compute_cost(weights, data, outputs)
                print("Epoch is:{} and Cost is: {}".format(epoch, cost))
                # stoppage criterion
                if abs(prev_cost - cost) < cost_threshold * prev_cost:
                    return weights
                prev_cost = cost
                nth += 1
               
    return weights

def sgd2(data, outputs, learning_rate = 0.0001, max_epochs = 4096, cost_threshold = 0.01):
   print("Training using Stochastic Gradient Descent 2 method")
   weights = np.random.uniform(-1.0, 1.0, data.shape[1])
   nth = 1
   prev_cost = 0.0
   # stochastic gradient descent
   for epoch in range(1, max_epochs):
      X, Y = shuffle(data, outputs)
      for x_chunk, y_chunk in zip(np.array_split(X, 100), np.array_split(Y, 100)):
         ascent = calculate_cost_gradient2(weights, x_chunk, y_chunk)
         weights = weights - learning_rate * ascent

      if epoch == 2 ** nth:
         cost = compute_cost2(weights, X, Y)
         print(f"Epoch is:{epoch} and Cost is: {cost}")
         # stoppage criterion
         if abs(prev_cost - cost) < cost_threshold * prev_cost:
            return weights
         prev_cost = cost
         nth += 1
   
   return weights

# Dataset

In [6]:
data = pd.read_csv('data_banknote_authentication.csv')

Y = data.iloc[:, -1]*2-1
X = data.iloc[:, 1:4]
X.insert(loc=len(X.columns), column='intercept', value=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

Question 4: Train and evaluate an SVC using the banknote_authentication data

In [7]:
# train the model using SGD1
# compute the time to calculate sgd

print("training with sgd()")
start_time = time.time()
W1 = sgd(X_train.to_numpy(), y_train.to_numpy(), cost_threshold=0.01, random_weights=True)
print("Training time for sgd(): %s seconds ---" % (time.time() - start_time))
print("training finished.")
print("weights are: {}".format(W1))


# testing the model on test set
sgd_predicted = []
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(X_test.to_numpy()[i], W1))
    sgd_predicted = np.append(sgd_predicted, yp)
print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), sgd_predicted)))


#train the model using SGD2
print("\ntraining with sgd2()")
start_time = time.time()
W2 = sgd2(X_train.to_numpy(), y_train.to_numpy(), cost_threshold=0.01)
print("Training time for sgd2(): %s seconds ---" % (time.time() - start_time))
print("training finished.")
print("weights are: {}".format(W2))


# testing the model on test set
sgd2_predicted = []
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(X_test.to_numpy()[i], W2))
    sgd2_predicted = np.append(sgd2_predicted, yp)
print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), sgd2_predicted)))

training with sgd()
Training using Stochastic Gradient Descent 1 method
Epoch is:1 and Cost is: 1365.1217913107123
Epoch is:2 and Cost is: 703.6417932095144
Epoch is:4 and Cost is: 483.6408403782307
Epoch is:8 and Cost is: 490.1201811982653
Epoch is:16 and Cost is: 524.5413041533213
Epoch is:32 and Cost is: 441.5748957071105
Epoch is:64 and Cost is: 593.1599946387327
Epoch is:128 and Cost is: 470.9467724498545
Epoch is:256 and Cost is: 475.2087574090331
Training time for sgd(): 0.291914701461792 seconds ---
training finished.
weights are: [-0.61613457 -0.39892281 -0.45951297  0.88392396]
accuracy on test dataset: 0.8214936247723132

training with sgd2()
Training using Stochastic Gradient Descent 2 method
Epoch is:2 and Cost is: 509.4665350995559
Epoch is:4 and Cost is: 656.3647361867406
Epoch is:8 and Cost is: 622.8166242966822
Epoch is:16 and Cost is: 548.8379896797907
Epoch is:32 and Cost is: 536.4863906211289
Epoch is:64 and Cost is: 505.32978523719675
Epoch is:128 and Cost is: 525.

In [8]:
def accuracy(y_true, y_pred):
    '''
    Define a function to test accuracy of the model
    '''
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

# This is using the reference code from AssemblyAI series on ML from scratch
clf = SupportVectorMachine()
start_time = time.time()
clf.fit(X_train.to_numpy(), y_train.to_numpy(), random_weights=True)
print("Training time for SupportVectorMachine(): %s seconds ---" % (time.time() - start_time))
predictions = clf.predict(X_test)
print (type(predictions))

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Training time for SupportVectorMachine(): 2.2477619647979736 seconds ---
<class 'numpy.ndarray'>
Accuracy: 0.8214936247723132


[Bonus] Question 5: Train and evaluate an SKLEARN SVC model, and compare the results to your model 

In [9]:
# Train a Support Vector Machine model from Sklearn using the same data as inputs
from sklearn.svm import SVC
#clf = SVC(kernel='linear')
clf = SVC(kernel='rbf') # I tried all different kernels and rbf gave the best accuracy
clf.fit(X_train, y_train)
sgd_predicted = clf.predict(X_test)
print("accuracy on test dataset using sklearn: {}".format(accuracy_score(y_test.to_numpy(), sgd_predicted)))


accuracy on test dataset using sklearn: 0.9016393442622951


Question 6: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

I feel like I learned the basics of SVM pretty well by implementing the SVM algorithm from scratch. My understanding of it (I have studied it a little bit before now as well.) is that it makes an attempt to find a hyperplane that divides the two classes of data points the best. It does this through a stochastic gradient descent algorithm. I still feel like I'm not entirely sure why exactly the loss function and the cost functions are best for this algorithm. Maybe I'll take that a little bit on faith because it appears to work well.

This week i actually have 3 slightly different implementations of stochastic gradient descent as part of the SVM algorithm. Really the heart of SVM is SGD.

1. The first implementation is loosely based on the resource provided by Chenwei in class. However that implementation didn't work. So I modified it until it worked.
2. The SVM class at the top of the file is the second implementation. It is probably the most straightforward implementation of SGD here.
3. I also included the SGD2() function, which is basically CHenwei's implementation.
4. Finally as the assignment directed, I trained the SVM model from sklearn. Unsurprisingly this performed best, but I had to use trial and error to find the best kernel function to achieve highest accuracy.

I wanted to compare the 3 algorithms that I implemented. As can be seen they all performed roughly the same on this weeks Bank NOte data set.

1. SGD() had accuracy = 82.14%
2. SGD2() had accuracy = 81.05%
3. the SVM class from Assembly AI had accuracy = 82.14% as well
4. Sklearn, being a professionally optimized library written in C++, of course had the best accuracy = 90.16%

Another interesting thing is the timing info gathered between SGD() and SGD2()

The time to train those two different models is quite different. SGD() is orders of magnitude faster than SGD2(). This comes down to one thing. I computed the hinge loss for an entire chunk up front using numpy vectorization. This is much faster than computing the hinge loss for each data point individually.

This optimization can be seen in the function ```calculate_cost_gradient```. In particular I calculate:

```
   # compute the losses up front
   # doing this outside the loop speeds up the gradient calculation
   losses = 1 - Y_batch * (np.dot(X_batch, W))
   losses[losses < 0] = 0  # convert anything less than 0 to 0
```
Then later use these losses when computing the gradient. This one change had a large positive performance impact. As can be seen in the timing info below.

```
   Training time for sgd(): 0.302901029586792 seconds
   Training time for sgd2(): 14.218122243881226 seconds
```

Another smaller optimization that had a positive impact on performance is initializing the Weights randomly instead of with all zeros. I added a parameter to both sgd() and sgd2() to use this optimization. In practice, I think it would always be better to have random weights instead of zeroed weights.